In [1]:
#Set working directory

#load libraries
library(gdata)
library(pander)
library(dplyr)
library(lme4)
library(afex)
library(lmerTest)
library(MuMIn)
library(Hmisc)

gdata: read.xls support for 'XLS' (Excel 97-2004) files ENABLED.



gdata: read.xls support for 'XLSX' (Excel 2007+) files ENABLED.


Attaching package: ‘gdata’


The following object is masked from ‘package:stats’:

    nobs


The following object is masked from ‘package:utils’:

    object.size


The following object is masked from ‘package:base’:

    startsWith



Attaching package: ‘dplyr’


The following objects are masked from ‘package:gdata’:

    combine, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: Matrix

************
Welcome to afex. For support visit: http://afex.singmann.science/

- Functions for ANOVAs: aov_car(), aov_ez(), and aov_4()
- Methods for calculating p-values with mixed(): 'KR', 'S', 'LRT', and 'PB'
- 'afex_aov' and 'mixed' objects can be passed to emmeans() for follow-up tests
- NEWS: library('em

In [2]:
AllPointingData = read.xls('Masterfile_Pointing_cleaned_noPilot.xlsx', na.strings=c("NA", "#NULL!")) 
#View(AllPointingData)
AllPointingData

participant,start_landmark,start_by_route,facing_landmark,target_landmark,target_by_route,bearing,actual_direction,abs_error,same_or_different,seenunseen,trial_type,error_start_RouteA,error_target_RouteA,error_start_RouteB,error_target_RouteB
<int>,<chr>,<int>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,Batty House,1,Lynch Station,Harvey House,1,65,93.194238,28.194238,same,1,1,28.194238,28.194238,NA,NA
1,Batty House,1,Lynch Station,Harris Hall,1,25,37.528637,12.528637,same,0,0,12.528637,12.528637,NA,NA
1,Batty House,1,Lynch Station,Tobler Museum,2,16,37.308022,21.308022,different,1,2,21.308022,NA,NA,21.308022
1,Batty House,1,Lynch Station,Snow Church,2,7,48.199476,41.199476,different,1,2,41.199476,NA,NA,41.199476
1,Batty House,1,Lynch Station,Sauer Center,2,14,30.331695,16.331695,different,1,2,16.331695,NA,NA,16.331695
1,Batty House,1,Lynch Station,Golledge Hall,2,25,51.767682,26.767682,different,1,2,26.767682,NA,NA,26.767682
1,Batty House,1,Lynch Station,Lynch Station,1,38,47.327984,9.327984,same,0,0,9.327984,9.327984,NA,NA
1,Lynch Station,1,Harris Hall,Harris Hall,1,2,7.021705,5.021705,same,0,0,5.021705,5.021705,NA,NA
1,Lynch Station,1,Harris Hall,Golledge Hall,2,23,3.323864,19.676136,different,1,2,19.676136,NA,NA,19.676136


In [3]:
dim(AllPointingData)

[1] 3613   16

In [4]:
length(unique(AllPointingData$participant)) #65 subs

[1] 65

In [5]:
###############################
#### Data processing #########
##############################

# In earlier submissions, for the pointing task, we do an ANOVA 
# with trial type (same vs. different route) as a within-participants factor, 
# and gender and route order as between-participants factors

# We want to construct the equivalent LMEMs with these factors
# But to do that that we need to bring over the gender and route order factors from some other file
# This information seems to be included in AllData_bysubs

# But first we need to figure out how many observations there are per participant
# There seem to be 56 trials per partcipant, with some trials missing

# Let's create a dataframe that shows the number of observations by participant
ParticipantTrials <- count(AllPointingData, participant)
#View(ParticipantTrials)
ParticipantTrials

participant,n
<int>,<int>
1,56
2,56
3,56
4,56
5,56
6,56
7,56
8,56
9,56


In [6]:
# Let's hold onto that column

# ParticipantTrials  <- ParticipantTrials[,2]

In [7]:
# OK, now let's grab the relevant demographic and design info from this file
# and repeat it according to the number of trials for each participant
# and finally combine this data structure with the AllPointingData df
BySubsData <- read.xls('AllData_bysubs.xlsx', na.strings=c("NA", "#NULL!")) 
#View(BySubsData)
head(BySubsData)

,Subject,sex,age,SBSOD_average,SBSOD_CY_average,Route_Order,RouteOrder_4,r2_Overall,r2_RouteA,r2_RouteB,⋯,error_start_routeA_SAME,error_start_routeB_SAME,error_target_routeA_SAME,error_target_routeB_SAME,error_start_RouteA_Different,error_target_RouteA_Different,error_start_RouteB_Different,error_target_RouteB_Different,low_SBSOD,high_SBSOD
,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>
1,1,1,21,3.000000,2.5,1,1,0.945822,0.826816,0.956720,⋯,25.72205,6.191736,25.72205,6.191736,19.78113,28.67941,20.30927,11.41099,1,NA
2,5,1,22,4.066667,4.2,1,1,0.505958,0.418005,0.956754,⋯,25.85940,13.154199,25.85940,13.154199,27.21885,49.21212,43.76703,21.77377,1,NA
3,8,1,21,3.533333,4.4,2,4,0.941772,0.597372,0.390829,⋯,45.90709,14.594749,45.90709,14.594749,44.21325,63.79064,50.37107,30.79368,1,NA
4,9,1,31,4.733333,4.5,1,1,0.169934,0.431045,0.250685,⋯,38.37047,15.725679,38.37047,15.725679,37.61004,55.89559,46.19068,27.90513,1,NA
5,10,1,19,3.933333,4.1,1,2,0.170237,0.662662,0.104487,⋯,44.82641,32.505434,44.82641,32.505434,46.35503,62.59099,57.31057,41.07461,1,NA
6,11,1,22,3.400000,3.7,2,3,0.670268,0.265635,0.248272,⋯,35.64581,42.896540,35.64581,42.896540,39.89112,50.86219,53.96964,42.99858,1,NA


In [8]:
#Let's change "Subject" into "participant" for consistency
names(BySubsData)[1]<-"participant"

In [9]:
#Let's sort this in ascending order to match ParticipantTrials
BySubsData <- BySubsData[order(BySubsData[,1]), ] #sort according to first column (Participant)

In [10]:
# Let's grab just the first 7 columns with the demographic / design info we need
DemographicData <- subset(BySubsData, select=1:7)
#View(DemographicData)
DemographicData

,participant,sex,age,SBSOD_average,SBSOD_CY_average,Route_Order,RouteOrder_4
,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<int>
1,1,1,21,3.000000,2.5,1,1
35,2,2,23,5.533333,5.2,1,2
36,3,1,21,4.533333,4.6,2,3
37,4,1,18,4.400000,4.8,2,4
2,5,1,22,4.066667,4.2,1,1
38,6,1,21,4.533333,5.3,1,2
39,7,1,23,5.000000,5.1,2,3
3,8,1,21,3.533333,4.4,2,4
4,9,1,31,4.733333,4.5,1,1


In [11]:
# Let's also grab the columns with the Space Syntax measures
which(colnames(BySubsData)=="Connectivity" ) #17h column
SpaceSyntaxMeasures <- subset(BySubsData, select=17:21)

[1] 17

In [12]:
# Let's combine the number of observations (n) the per participant, the DemographicData, and the SpaceSyntax measures
CombinedData <-cbind(DemographicData, ParticipantTrials, SpaceSyntaxMeasures)

In [13]:
AllPointingData = read.xls('Masterfile_Pointing_cleaned_noPilot.xlsx', na.strings=c("NA", "#NULL!")) 
ParticipantTrials <- count(AllPointingData, participant)

In [14]:
# Let's repeat each row by n times and create an expanded df
Combined_expanded <- CombinedData[rep(rownames(CombinedData), CombinedData$n), ]

In [15]:
#View(Combined_expanded)
nrow(Combined_expanded) #OK cool, we now have 3613 rows as our Pointing file

[1] 3613

In [16]:
#Before merging them let's check how Subjects are organized in the Pointing file (ascending order?)
#View(AllPointingData)
# Nope, it goes 1, 10, 11, etc. So let's reorganize by the first column (Participant)
AllPointingData <- AllPointingData[order(AllPointingData[,1]), ] #sort according to first column (Participant)

In [17]:
# Now let's combine with the expanded Demographics / Design info
AllPointingData <- cbind(Combined_expanded,AllPointingData[,2:16])

In [18]:
AllPointingData

,participant,sex,age,SBSOD_average,SBSOD_CY_average,Route_Order,RouteOrder_4,participant.1,n,Connectivity,⋯,bearing,actual_direction,abs_error,same_or_different,seenunseen,trial_type,error_start_RouteA,error_target_RouteA,error_start_RouteB,error_target_RouteB
,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<dbl>,⋯,<int>,<dbl>,<dbl>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,1,21,3,2.5,1,1,1,56,3.71429,⋯,65,93.194238,28.194238,same,1,1,28.194238,28.194238,NA,NA
1.1,1,1,21,3,2.5,1,1,1,56,3.71429,⋯,25,37.528637,12.528637,same,0,0,12.528637,12.528637,NA,NA
1.2,1,1,21,3,2.5,1,1,1,56,3.71429,⋯,16,37.308022,21.308022,different,1,2,21.308022,NA,NA,21.308022
1.3,1,1,21,3,2.5,1,1,1,56,3.71429,⋯,7,48.199476,41.199476,different,1,2,41.199476,NA,NA,41.199476
1.4,1,1,21,3,2.5,1,1,1,56,3.71429,⋯,14,30.331695,16.331695,different,1,2,16.331695,NA,NA,16.331695
1.5,1,1,21,3,2.5,1,1,1,56,3.71429,⋯,25,51.767682,26.767682,different,1,2,26.767682,NA,NA,26.767682
1.6,1,1,21,3,2.5,1,1,1,56,3.71429,⋯,38,47.327984,9.327984,same,0,0,9.327984,9.327984,NA,NA
1.7,1,1,21,3,2.5,1,1,1,56,3.71429,⋯,2,7.021705,5.021705,same,0,0,5.021705,5.021705,NA,NA
1.8,1,1,21,3,2.5,1,1,1,56,3.71429,⋯,23,3.323864,19.676136,different,1,2,19.676136,NA,NA,19.676136


In [19]:
######################
#### Sanity check ####
######################

# Let's reproduce the aggregate error variables (contained in the BySubsData file) from the AllPointingData data
# Aggregating by "participant" column name "abs_error"
MeanAbsError <- aggregate(AllPointingData[, c("abs_error")], list(AllPointingData$participant), mean)
colnames(MeanAbsError) = c("participant2", "abs_error")

In [20]:
str(AllPointingData)
which(colnames(AllPointingData)=="error_start_RouteA" ) #25th column
# Let's aggregate by participant columns 20-23 (error_start_RouteA, error_target_RouteA, error_start_RouteB, error_target_RouteB)

'data.frame':	3613 obs. of  29 variables:
 $ participant        : int  1 1 1 1 1 1 1 1 1 1 ...
 $ sex                : int  1 1 1 1 1 1 1 1 1 1 ...
 $ age                : int  21 21 21 21 21 21 21 21 21 21 ...
 $ SBSOD_average      : num  3 3 3 3 3 3 3 3 3 3 ...
 $ SBSOD_CY_average   : num  2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.5 ...
 $ Route_Order        : int  1 1 1 1 1 1 1 1 1 1 ...
 $ RouteOrder_4       : int  1 1 1 1 1 1 1 1 1 1 ...
 $ participant        : int  1 1 1 1 1 1 1 1 1 1 ...
 $ n                  : int  56 56 56 56 56 56 56 56 56 56 ...
 $ Connectivity       : num  3.71 3.71 3.71 3.71 3.71 ...
 $ Integration_Mean   : num  1.4 1.4 1.4 1.4 1.4 ...
 $ IntegrationR3_Mean : num  1.66 1.66 1.66 1.66 1.66 ...
 $ Depth_Mean         : num  2.59 2.59 2.59 2.59 2.59 ...
 $ DepthR3_Mean       : num  2.12 2.12 2.12 2.12 2.12 ...
 $ start_landmark     : chr  "Batty House" "Batty House" "Batty House" "Batty House" ...
 $ start_by_route     : int  1 1 1 1 1 1 1 1 1 1 ...
 $ facing_land

[1] 26

In [21]:
MeanStartTargetErrors <- aggregate(AllPointingData[, 25:28], list(AllPointingData$participant), mean,  na.rm=TRUE, na.action=NULL)
#View(MeanStartTargetErrors)
MeanStartTargetErrors

Group.1,trial_type,error_start_RouteA,error_target_RouteA,error_start_RouteB
<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,1.267857,19.78113,28.67941,20.309270
2,1.267857,27.25540,28.84310,27.476587
3,1.267857,25.08009,48.14568,42.366709
4,1.267857,29.65694,27.67370,20.653024
5,1.267857,27.21885,49.21212,43.767034
6,1.267857,34.13608,52.75195,40.701161
7,1.267857,22.29536,44.46375,36.992285
8,1.267857,44.21325,63.79064,50.371065
9,1.267857,37.61004,55.89559,46.190677


In [22]:
MeanErrors <- cbind(MeanAbsError,MeanStartTargetErrors[,2:5]) # OK, good the mean errors are reproducible with what we have in the BySubs file

In [23]:
# Compare the computed errors in MeanErrors with those in BySubsData
#View(MeanErrors)
MeanErrors

participant2,abs_error,trial_type,error_start_RouteA,error_target_RouteA,error_start_RouteB
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,20.04520,1.267857,19.78113,28.67941,20.309270
2,27.36599,1.267857,27.25540,28.84310,27.476587
3,33.72340,1.267857,25.08009,48.14568,42.366709
4,25.15498,1.267857,29.65694,27.67370,20.653024
5,35.49294,1.267857,27.21885,49.21212,43.767034
6,37.41862,1.267857,34.13608,52.75195,40.701161
7,29.64382,1.267857,22.29536,44.46375,36.992285
8,47.29216,1.267857,44.21325,63.79064,50.371065
9,41.90036,1.267857,37.61004,55.89559,46.190677


In [24]:
#View(BySubsData) #they're the same. good.
BySubsData

,participant,sex,age,SBSOD_average,SBSOD_CY_average,Route_Order,RouteOrder_4,r2_Overall,r2_RouteA,r2_RouteB,⋯,error_start_routeA_SAME,error_start_routeB_SAME,error_target_routeA_SAME,error_target_routeB_SAME,error_start_RouteA_Different,error_target_RouteA_Different,error_start_RouteB_Different,error_target_RouteB_Different,low_SBSOD,high_SBSOD
,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>
1,1,1,21,3.000000,2.5,1,1,0.945822,0.826816,0.956720,⋯,25.72205,6.191736,25.72205,6.191736,19.78113,28.67941,20.309270,11.41099,1,NA
35,2,2,23,5.533333,5.2,1,2,0.146714,0.666080,0.778289,⋯,13.86284,10.674301,13.86284,10.674301,27.25540,28.84311,27.476587,25.88888,NA,2
36,3,1,21,4.533333,4.6,2,3,0.282986,0.554284,0.333108,⋯,25.25847,11.774200,25.25847,11.774200,25.08009,48.14568,42.366709,19.30112,NA,2
37,4,1,18,4.400000,4.8,2,4,0.361452,0.388110,0.915188,⋯,23.92846,7.546892,23.92846,7.546892,29.65694,27.67370,20.653024,22.63626,NA,2
2,5,1,22,4.066667,4.2,1,1,0.505958,0.418005,0.956754,⋯,25.85940,13.154199,25.85940,13.154199,27.21885,49.21212,43.767034,21.77377,1,NA
38,6,1,21,4.533333,5.3,1,2,0.382166,0.475093,0.710581,⋯,33.46825,5.349743,33.46825,5.349743,34.13608,52.75195,40.701161,22.08529,NA,2
39,7,1,23,5.000000,5.1,2,3,0.297213,0.334312,0.706555,⋯,24.20417,6.770745,24.20417,6.770745,22.29536,44.46375,36.992285,14.82390,NA,2
3,8,1,21,3.533333,4.4,2,4,0.941772,0.597372,0.390829,⋯,45.90709,14.594749,45.90709,14.594749,44.21325,63.79064,50.371065,30.79368,1,NA
4,9,1,31,4.733333,4.5,1,1,0.169934,0.431045,0.250685,⋯,38.37047,15.725679,38.37047,15.725679,37.61004,55.89559,46.190677,27.90513,1,NA


In [25]:
##################################################
### Check observation numbers for trial types ###
##################################################

table(AllPointingData$same_or_different)


different      same 
     2067      1546 

In [26]:
# 2 = different, 1 = same-seen, 2 = same-unseen
table(AllPointingData$trial_type)


   0    1    2 
1094  452 2067 

In [27]:
####################################################
#### Adding Axial Map metrics for each landmark ####
####################################################

# AxialMetrics are contained in this Excel file
AxialMapMetrics = read.xls('Axial_Map_Analyses_by_Landmark_35_nodes_system.xlsx', na.strings=c("NA", "#NULL!")) 
head(AxialMapMetrics, n=8)
#table(AxialMapMetrics$Landmark)

,Ref,Landmark,Choice,Connectivity,Integration..HH.,Route
,<int>,<chr>,<int>,<int>,<dbl>,<chr>
1,27,Batty House,12,2,0.9475222,A
2,27,Lynch Station,12,2,0.9475222,A
3,26,Harris Hall,95,3,1.3078476,A
4,25,Harvey House,141,4,1.2380958,A
5,11,Sauer Center,59,6,1.5476197,B
6,12,Golledge Hall,234,7,1.7857150,B
7,14,Tobler Museum,3,3,1.2548268,B
8,20,Snow Church,132,6,1.4739234,B


In [28]:
#Let's check the distribution of landmarks as start points and as targets
table(AllPointingData$start_landmark)
table(AllPointingData$target_landmark)


  Batty House Golledge Hall   Harris Hall  Harvey House Lynch Station 
          442           455           453           452           453 
 Sauer Center   Snow Church Tobler Museum 
          454           449           455 


  Batty House Golledge Hall   Harris Hall  Harvey House Lynch Station 
          454           451           451           451           451 
 Sauer Center   Snow Church Tobler Museum 
          451           453           451 

In [29]:
#Let's start by just assigning the start and target landmark to variables for connectivity
AllPointingData$Connectivity_start_landmark <-AllPointingData$start_landmark 
AllPointingData$Connectivity_target_landmark <- AllPointingData$target_landmark

In [30]:
#Let's do the same for integration
AllPointingData$Integration_start_landmark <- AllPointingData$start_landmark 
AllPointingData$Integration_target_landmark <- AllPointingData$target_landmark

In [31]:
#Let's do the same for choice
AllPointingData$Choice_start_landmark <- AllPointingData$start_landmark 
AllPointingData$Choice_target_landmark <- AllPointingData$target_landmark

In [32]:
# TO CORRECT THE ERROR FOR THE DOUBLE COLUMN IN THE NEXT CELL
AllPointingData[8] <- list(NULL)

In [33]:
# Now let's assign each landmark its Connectivity value "Error: Can't transform a data frame with duplicate names"

AllPointingData <- AllPointingData %>%
mutate_at(.vars = vars("Connectivity_start_landmark", "Connectivity_target_landmark"),
          .funs  = funs(recode(., "Batty House" = 2, 
                               "Lynch Station" = 2, 
                               "Harris Hall" = 3, 
                               "Harvey House" = 4,  
                               "Sauer Center" = 6,
                               "Golledge Hall" = 7, 
                               "Tobler Museum" = 3, 
                               "Snow Church" = 6, 
                               .default = NA_real_))) 

Warning message:
“`funs()` was deprecated in dplyr 0.8.0.
Please use a list of either functions or lambdas: 

  # Simple named list: 
  list(mean = mean, median = median)

  # Auto named with `tibble::lst()`: 
  tibble::lst(mean, median)

  # Using lambdas
  list(~ mean(., trim = .2), ~ median(., na.rm = TRUE))
This warning is displayed once every 8 hours.
Call `lifecycle::last_warnings()` to see where this warning was generated.”


In [34]:
# Now let's assign each landmark its Connectivity value (corrected)
AllPointingData$Connectivity_start_landmark[AllPointingData$Connectivity_start_landmark == "Batty House"] <- 2
AllPointingData$Connectivity_start_landmark[AllPointingData$Connectivity_start_landmark == "Lynch Station"] <- 2
AllPointingData$Connectivity_start_landmark[AllPointingData$Connectivity_start_landmark == "Harris Hall"] <- 3
AllPointingData$Connectivity_start_landmark[AllPointingData$Connectivity_start_landmark == "Harvey House"] <- 4
AllPointingData$Connectivity_start_landmark[AllPointingData$Connectivity_start_landmark == "Sauer Center"] <- 6
AllPointingData$Connectivity_start_landmark[AllPointingData$Connectivity_start_landmark == "Golledge Hall"] <- 7
AllPointingData$Connectivity_start_landmark[AllPointingData$Connectivity_start_landmark == "Tobler Museum"] <- 3
AllPointingData$Connectivity_start_landmark[AllPointingData$Connectivity_start_landmark == "Snow Church"] <- 6
AllPointingData$Connectivity_start_landmark[AllPointingData$Connectivity_start_landmark == NA] <- 0

AllPointingData$Connectivity_target_landmark[AllPointingData$Connectivity_target_landmark == "Batty House"] <- 2
AllPointingData$Connectivity_target_landmark[AllPointingData$Connectivity_target_landmark == "Lynch Station"] <- 2
AllPointingData$Connectivity_target_landmark[AllPointingData$Connectivity_target_landmark == "Harris Hall"] <- 3
AllPointingData$Connectivity_target_landmark[AllPointingData$Connectivity_target_landmark == "Harvey House"] <- 4
AllPointingData$Connectivity_target_landmark[AllPointingData$Connectivity_target_landmark == "Sauer Center"] <- 6
AllPointingData$Connectivity_target_landmark[AllPointingData$Connectivity_target_landmark == "Golledge Hall"] <- 7
AllPointingData$Connectivity_target_landmark[AllPointingData$Connectivity_target_landmark == "Tobler Museum"] <- 3
AllPointingData$Connectivity_target_landmark[AllPointingData$Connectivity_target_landmark == "Snow Church"] <- 6
AllPointingData$Connectivity_target_landmark[AllPointingData$Connectivity_target_landmark == NA] <- 0

In [35]:
# Now let's assign each landmark its Integration value (corrected)

AllPointingData$Integration_start_landmark[AllPointingData$Integration_start_landmark == "Batty House"] <- 0.94752222
AllPointingData$Integration_start_landmark[AllPointingData$Integration_start_landmark == "Lynch Station"] <- 0.94752222
AllPointingData$Integration_start_landmark[AllPointingData$Integration_start_landmark == "Harris Hall"] <- 1.3078476
AllPointingData$Integration_start_landmark[AllPointingData$Integration_start_landmark == "Harvey House"] <- 1.2380958
AllPointingData$Integration_start_landmark[AllPointingData$Integration_start_landmark == "Sauer Center"] <- 1.5476197
AllPointingData$Integration_start_landmark[AllPointingData$Integration_start_landmark == "Golledge Hall"] <- 1.785715
AllPointingData$Integration_start_landmark[AllPointingData$Integration_start_landmark == "Tobler Museum"] <- 1.2548268
AllPointingData$Integration_start_landmark[AllPointingData$Integration_start_landmark == "Snow Church"] <- 1.4739234
AllPointingData$Integration_start_landmark[AllPointingData$Integration_start_landmark == NA] <- 0

AllPointingData$Integration_target_landmark[AllPointingData$Integration_target_landmark == "Batty House"] <- 0.94752222
AllPointingData$Integration_target_landmark[AllPointingData$Integration_target_landmark == "Lynch Station"] <- 0.94752222
AllPointingData$Integration_target_landmark[AllPointingData$Integration_target_landmark == "Harris Hall"] <- 1.3078476
AllPointingData$Integration_target_landmark[AllPointingData$Integration_target_landmark == "Harvey House"] <- 1.2380958
AllPointingData$Integration_target_landmark[AllPointingData$Integration_target_landmark == "Sauer Center"] <- 1.5476197
AllPointingData$Integration_target_landmark[AllPointingData$Integration_target_landmark == "Golledge Hall"] <- 1.785715
AllPointingData$Integration_target_landmark[AllPointingData$Integration_target_landmark == "Tobler Museum"] <- 1.2548268
AllPointingData$Integration_target_landmark[AllPointingData$Integration_target_landmark == "Snow Church"] <- 1.4739234
AllPointingData$Integration_target_landmark[AllPointingData$Integration_target_landmark == NA] <- 0

In [36]:
# Now let's assign each landmark its Choice value (corrected)

AllPointingData$Choice_start_landmark[AllPointingData$Choice_start_landmark == "Batty House"] <- 12
AllPointingData$Choice_start_landmark[AllPointingData$Choice_start_landmark == "Lynch Station"] <- 12
AllPointingData$Choice_start_landmark[AllPointingData$Choice_start_landmark == "Harris Hall"] <- 95
AllPointingData$Choice_start_landmark[AllPointingData$Choice_start_landmark == "Harvey House"] <- 141
AllPointingData$Choice_start_landmark[AllPointingData$Choice_start_landmark == "Sauer Center"] <- 59
AllPointingData$Choice_start_landmark[AllPointingData$Choice_start_landmark == "Golledge Hall"] <- 234
AllPointingData$Choice_start_landmark[AllPointingData$Choice_start_landmark == "Tobler Museum"] <- 3
AllPointingData$Choice_start_landmark[AllPointingData$Choice_start_landmark == "Snow Church"] <- 132
AllPointingData$Choice_start_landmark[AllPointingData$Choice_start_landmark == NA] <- 0

AllPointingData$Choice_target_landmark[AllPointingData$Choice_target_landmark == "Batty House"] <- 12
AllPointingData$Choice_target_landmark[AllPointingData$Choice_target_landmark == "Lynch Station"] <- 12
AllPointingData$Choice_target_landmark[AllPointingData$Choice_target_landmark == "Harris Hall"] <- 95
AllPointingData$Choice_target_landmark[AllPointingData$Choice_target_landmark == "Harvey House"] <- 141
AllPointingData$Choice_target_landmark[AllPointingData$Choice_target_landmark == "Sauer Center"] <- 59
AllPointingData$Choice_target_landmark[AllPointingData$Choice_target_landmark == "Golledge Hall"] <- 234
AllPointingData$Choice_target_landmark[AllPointingData$Choice_target_landmark == "Tobler Museum"] <- 3
AllPointingData$Choice_target_landmark[AllPointingData$Choice_target_landmark == "Snow Church"] <- 132
AllPointingData$Choice_target_landmark[AllPointingData$Choice_target_landmark == NA] <- 0

In [37]:
head(AllPointingData)

,participant,sex,age,SBSOD_average,SBSOD_CY_average,Route_Order,RouteOrder_4,n,Connectivity,Integration_Mean,⋯,error_start_RouteA,error_target_RouteA,error_start_RouteB,error_target_RouteB,Connectivity_start_landmark,Connectivity_target_landmark,Integration_start_landmark,Integration_target_landmark,Choice_start_landmark,Choice_target_landmark
,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1,1,1,21,3,2.5,1,1,56,3.71429,1.39777,⋯,28.19424,28.19424,NA,NA,2,4,0.94752222,1.2380958,12,141
1.1,1,1,21,3,2.5,1,1,56,3.71429,1.39777,⋯,12.52864,12.52864,NA,NA,2,3,0.94752222,1.3078476,12,95
1.2,1,1,21,3,2.5,1,1,56,3.71429,1.39777,⋯,21.30802,NA,NA,21.30802,2,3,0.94752222,1.2548268,12,3
1.3,1,1,21,3,2.5,1,1,56,3.71429,1.39777,⋯,41.19948,NA,NA,41.19948,2,6,0.94752222,1.4739234,12,132
1.4,1,1,21,3,2.5,1,1,56,3.71429,1.39777,⋯,16.33169,NA,NA,16.33169,2,6,0.94752222,1.5476197,12,59
1.5,1,1,21,3,2.5,1,1,56,3.71429,1.39777,⋯,26.76768,NA,NA,26.76768,2,7,0.94752222,1.785715,12,234


In [38]:
##############################################
#### Adding VGA metrics for each landmark ####
##############################################

# VGA metrics contained in this Excel file
VGAMetrics = read.xls('Visibility_Graph_Analyses_by_Landmark_35_nodes_system.xlsx', na.strings=c("NA", "#NULL!")) 
head(VGAMetrics, n=8)

,Landmark,Connectivity,Visual.Integration..HH.,Through.Vision,Route
,<chr>,<dbl>,<dbl>,<dbl>,<chr>
1,Batty House,101.1310,2.09182,1334.650,A
2,Lynch Station,106.0000,2.19206,778.918,A
3,Harris Hall,100.8160,2.53761,1001.240,A
4,Harvey House,68.9032,2.42301,287.387,A
5,Sauer Center,64.4815,2.44747,286.222,B
6,Golledge Hall,108.5370,2.90956,4539.110,B
7,Tobler Museum,80.0175,2.71935,522.333,B
8,Snow Church,94.5893,2.60978,658.607,B


In [39]:
# Let's start by just assigning the start and target landmark to variables for connectivity
AllPointingData$VGA_Connectivity_start_landmark <-AllPointingData$start_landmark 
AllPointingData$VGA_Connectivity_target_landmark <- AllPointingData$target_landmark

In [40]:
# Let's do the same for integration
AllPointingData$VGA_Integration_start_landmark <- AllPointingData$start_landmark 
AllPointingData$VGA_Integration_target_landmark <- AllPointingData$target_landmark

In [41]:
# And let's do the same for Through Vision
AllPointingData$VGA_ThruVision_start_landmark <- AllPointingData$start_landmark 
AllPointingData$VGA_ThruVision_target_landmark <- AllPointingData$target_landmark

In [42]:
# Now let's assign each landmark its VGA-Connectivity value (corrected)

AllPointingData$VGA_Connectivity_start_landmark[AllPointingData$VGA_Connectivity_start_landmark == "Batty House"] <- 101.1310
AllPointingData$VGA_Connectivity_start_landmark[AllPointingData$VGA_Connectivity_start_landmark == "Lynch Station"] <- 106.0000
AllPointingData$VGA_Connectivity_start_landmark[AllPointingData$VGA_Connectivity_start_landmark == "Harris Hall"] <- 100.8160
AllPointingData$VGA_Connectivity_start_landmark[AllPointingData$VGA_Connectivity_start_landmark == "Harvey House"] <- 68.9032
AllPointingData$VGA_Connectivity_start_landmark[AllPointingData$VGA_Connectivity_start_landmark == "Sauer Center"] <- 64.4815
AllPointingData$VGA_Connectivity_start_landmark[AllPointingData$VGA_Connectivity_start_landmark == "Golledge Hall"] <- 108.5370
AllPointingData$VGA_Connectivity_start_landmark[AllPointingData$VGA_Connectivity_start_landmark == "Tobler Museum"] <- 80.0175
AllPointingData$VGA_Connectivity_start_landmark[AllPointingData$VGA_Connectivity_start_landmark == "Snow Church"] <- 94.5893
AllPointingData$VGA_Connectivity_start_landmark[AllPointingData$VGA_Connectivity_start_landmark == NA] <- 0

AllPointingData$VGA_Connectivity_target_landmark[AllPointingData$VGA_Connectivity_target_landmark == "Batty House"] <- 101.1310
AllPointingData$VGA_Connectivity_target_landmark[AllPointingData$VGA_Connectivity_target_landmark == "Lynch Station"] <- 106.0000
AllPointingData$VGA_Connectivity_target_landmark[AllPointingData$VGA_Connectivity_target_landmark == "Harris Hall"] <- 100.8160
AllPointingData$VGA_Connectivity_target_landmark[AllPointingData$VGA_Connectivity_target_landmark == "Harvey House"] <- 68.9032
AllPointingData$VGA_Connectivity_target_landmark[AllPointingData$VGA_Connectivity_target_landmark == "Sauer Center"] <- 64.4815
AllPointingData$VGA_Connectivity_target_landmark[AllPointingData$VGA_Connectivity_target_landmark == "Golledge Hall"] <- 108.5370
AllPointingData$VGA_Connectivity_target_landmark[AllPointingData$VGA_Connectivity_target_landmark == "Tobler Museum"] <- 80.0175
AllPointingData$VGA_Connectivity_target_landmark[AllPointingData$VGA_Connectivity_target_landmark == "Snow Church"] <- 94.5893
AllPointingData$VGA_Connectivity_target_landmark[AllPointingData$VGA_Connectivity_target_landmark == NA] <- 0

In [43]:
# Now let's assign each landmark its VGA-Integration value (corrected)

AllPointingData$VGA_Integration_start_landmark[AllPointingData$VGA_Integration_start_landmark == "Batty House"] <- 2.09182
AllPointingData$VGA_Integration_start_landmark[AllPointingData$VGA_Integration_start_landmark == "Lynch Station"] <- 2.19206
AllPointingData$VGA_Integration_start_landmark[AllPointingData$VGA_Integration_start_landmark == "Harris Hall"] <- 2.53761
AllPointingData$VGA_Integration_start_landmark[AllPointingData$VGA_Integration_start_landmark == "Harvey House"] <- 2.42301
AllPointingData$VGA_Integration_start_landmark[AllPointingData$VGA_Integration_start_landmark == "Sauer Center"] <- 2.44747
AllPointingData$VGA_Integration_start_landmark[AllPointingData$VGA_Integration_start_landmark == "Golledge Hall"] <- 2.90956
AllPointingData$VGA_Integration_start_landmark[AllPointingData$VGA_Integration_start_landmark == "Tobler Museum"] <- 2.71935
AllPointingData$VGA_Integration_start_landmark[AllPointingData$VGA_Integration_start_landmark == "Snow Church"] <- 2.60978
AllPointingData$VGA_Integration_start_landmark[AllPointingData$VGA_Integration_start_landmark == NA] <- 0

AllPointingData$VGA_Integration_target_landmark[AllPointingData$VGA_Integration_target_landmark == "Batty House"] <- 2.09182
AllPointingData$VGA_Integration_target_landmark[AllPointingData$VGA_Integration_target_landmark == "Lynch Station"] <- 2.19206
AllPointingData$VGA_Integration_target_landmark[AllPointingData$VGA_Integration_target_landmark == "Harris Hall"] <- 2.53761
AllPointingData$VGA_Integration_target_landmark[AllPointingData$VGA_Integration_target_landmark == "Harvey House"] <- 2.42301
AllPointingData$VGA_Integration_target_landmark[AllPointingData$VGA_Integration_target_landmark == "Sauer Center"] <- 2.44747
AllPointingData$VGA_Integration_target_landmark[AllPointingData$VGA_Integration_target_landmark == "Golledge Hall"] <- 2.90956
AllPointingData$VGA_Integration_target_landmark[AllPointingData$VGA_Integration_target_landmark == "Tobler Museum"] <- 2.71935
AllPointingData$VGA_Integration_target_landmark[AllPointingData$VGA_Integration_target_landmark == "Snow Church"] <- 2.60978
AllPointingData$VGA_Integration_target_landmark[AllPointingData$VGA_Integration_target_landmark == NA] <- 0

In [44]:
# Now let's assign each landmark its VGA-Integration value (corrected)

AllPointingData$VGA_ThruVision_start_landmark[AllPointingData$VGA_ThruVision_start_landmark == "Batty House"] <- 1334.65
AllPointingData$VGA_ThruVision_start_landmark[AllPointingData$VGA_ThruVision_start_landmark == "Lynch Station"] <- 778.918
AllPointingData$VGA_ThruVision_start_landmark[AllPointingData$VGA_ThruVision_start_landmark == "Harris Hall"] <- 1001.24
AllPointingData$VGA_ThruVision_start_landmark[AllPointingData$VGA_ThruVision_start_landmark == "Harvey House"] <- 287.387
AllPointingData$VGA_ThruVision_start_landmark[AllPointingData$VGA_ThruVision_start_landmark == "Sauer Center"] <- 286.222
AllPointingData$VGA_ThruVision_start_landmark[AllPointingData$VGA_ThruVision_start_landmark == "Golledge Hall"] <- 4539.11
AllPointingData$VGA_ThruVision_start_landmark[AllPointingData$VGA_ThruVision_start_landmark == "Tobler Museum"] <- 522.333
AllPointingData$VGA_ThruVision_start_landmark[AllPointingData$VGA_ThruVision_start_landmark == "Snow Church"] <- 658.607
AllPointingData$VGA_ThruVision_start_landmark[AllPointingData$VGA_ThruVision_start_landmark == NA] <- 0

AllPointingData$VGA_ThruVision_target_landmark[AllPointingData$VGA_ThruVision_target_landmark == "Batty House"] <- 1334.65
AllPointingData$VGA_ThruVision_target_landmark[AllPointingData$VGA_ThruVision_target_landmark == "Lynch Station"] <- 778.918
AllPointingData$VGA_ThruVision_target_landmark[AllPointingData$VGA_ThruVision_target_landmark == "Harris Hall"] <- 1001.24
AllPointingData$VGA_ThruVision_target_landmark[AllPointingData$VGA_ThruVision_target_landmark == "Harvey House"] <- 287.387
AllPointingData$VGA_ThruVision_target_landmark[AllPointingData$VGA_ThruVision_target_landmark == "Sauer Center"] <- 286.222
AllPointingData$VGA_ThruVision_target_landmark[AllPointingData$VGA_ThruVision_target_landmark == "Golledge Hall"] <- 4539.11
AllPointingData$VGA_ThruVision_target_landmark[AllPointingData$VGA_ThruVision_target_landmark == "Tobler Museum"] <- 522.333
AllPointingData$VGA_ThruVision_target_landmark[AllPointingData$VGA_ThruVision_target_landmark == "Snow Church"] <- 658.607
AllPointingData$VGA_ThruVision_target_landmark[AllPointingData$VGA_ThruVision_target_landmark == NA] <- 0

In [45]:
head(AllPointingData)

,participant,sex,age,SBSOD_average,SBSOD_CY_average,Route_Order,RouteOrder_4,n,Connectivity,Integration_Mean,⋯,Integration_start_landmark,Integration_target_landmark,Choice_start_landmark,Choice_target_landmark,VGA_Connectivity_start_landmark,VGA_Connectivity_target_landmark,VGA_Integration_start_landmark,VGA_Integration_target_landmark,VGA_ThruVision_start_landmark,VGA_ThruVision_target_landmark
,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,1,1,21,3,2.5,1,1,56,3.71429,1.39777,⋯,0.94752222,1.2380958,12,141,101.131,68.9032,2.09182,2.42301,1334.65,287.387
1.1,1,1,21,3,2.5,1,1,56,3.71429,1.39777,⋯,0.94752222,1.3078476,12,95,101.131,100.816,2.09182,2.53761,1334.65,1001.24
1.2,1,1,21,3,2.5,1,1,56,3.71429,1.39777,⋯,0.94752222,1.2548268,12,3,101.131,80.0175,2.09182,2.71935,1334.65,522.333
1.3,1,1,21,3,2.5,1,1,56,3.71429,1.39777,⋯,0.94752222,1.4739234,12,132,101.131,94.5893,2.09182,2.60978,1334.65,658.607
1.4,1,1,21,3,2.5,1,1,56,3.71429,1.39777,⋯,0.94752222,1.5476197,12,59,101.131,64.4815,2.09182,2.44747,1334.65,286.222
1.5,1,1,21,3,2.5,1,1,56,3.71429,1.39777,⋯,0.94752222,1.785715,12,234,101.131,108.537,2.09182,2.90956,1334.65,4539.11


In [46]:
################################################
#### Some descriptives about Pointing Error ####
################################################

pander(aggregate(abs_error~same_or_different,data=AllPointingData,FUN=mean)) 
pander(aggregate(abs_error~same_or_different,data=AllPointingData,FUN=sd))



| same_or_different | abs_error |
|:-----------------:|:---------:|
|     different     |   48.3    |
|       same        |   28.51   |



| same_or_different | abs_error |
|:-----------------:|:---------:|
|     different     |   41.75   |
|       same        |   32.9    |



In [47]:
# Alternatively, with dplyer
summtError = dplyr::summarize(group_by(AllPointingData,same_or_different),
                              mean=mean(abs_error, na.rm = TRUE), sd= sd(abs_error, na.rm = TRUE), se=sd(abs_error, na.rm = TRUE)/sqrt(length(abs_error)))

print.data.frame(summtError)

summtError = dplyr::summarize(group_by(AllPointingData,sex),
                              mean=mean(abs_error, na.rm = TRUE), sd= sd(abs_error, na.rm = TRUE), se=sd(abs_error, na.rm = TRUE)/sqrt(length(abs_error)))
print.data.frame(summtError)

  same_or_different     mean       sd        se
1         different 48.30185 41.74828 0.9182657
2              same 28.50757 32.89615 0.8366433
  sex     mean       sd        se
1   1 41.24370 39.73761 0.7853835
2   2 36.39959 38.52042 1.1870714


In [48]:
#############################################
######## Pointing task LMEMs  ################
#############################################

# Let's check if our variables are of the right type and define them as needed
str(AllPointingData)

AllPointingData$participant <- as.factor(as.matrix(AllPointingData$participant))
AllPointingData$same_or_different <- as.factor(as.matrix(AllPointingData$same_or_different))
AllPointingData$seenunseen <- as.factor(as.matrix(AllPointingData$seenunseen))
AllPointingData$sex <- as.factor(as.matrix(AllPointingData$sex)) # Marianna, which is sex 1 and sex 2?
AllPointingData$Route_Order <- as.factor(as.matrix(AllPointingData$Route_Order)) # Marianna, is this the order variable we entered in ANOVAs?

'data.frame':	3613 obs. of  40 variables:
 $ participant                     : int  1 1 1 1 1 1 1 1 1 1 ...
 $ sex                             : int  1 1 1 1 1 1 1 1 1 1 ...
 $ age                             : int  21 21 21 21 21 21 21 21 21 21 ...
 $ SBSOD_average                   : num  3 3 3 3 3 3 3 3 3 3 ...
 $ SBSOD_CY_average                : num  2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.5 ...
 $ Route_Order                     : int  1 1 1 1 1 1 1 1 1 1 ...
 $ RouteOrder_4                    : int  1 1 1 1 1 1 1 1 1 1 ...
 $ n                               : int  56 56 56 56 56 56 56 56 56 56 ...
 $ Connectivity                    : num  3.71 3.71 3.71 3.71 3.71 ...
 $ Integration_Mean                : num  1.4 1.4 1.4 1.4 1.4 ...
 $ IntegrationR3_Mean              : num  1.66 1.66 1.66 1.66 1.66 ...
 $ Depth_Mean                      : num  2.59 2.59 2.59 2.59 2.59 ...
 $ DepthR3_Mean                    : num  2.12 2.12 2.12 2.12 2.12 ...
 $ start_landmark                  : chr

In [49]:
## Axial Map Analysis predictors 
AllPointingData$Connectivity_start_landmark <- as.numeric(as.matrix(AllPointingData$Connectivity_start_landmark))
AllPointingData$Connectivity_target_landmark <- as.numeric(as.matrix(AllPointingData$Connectivity_target_landmark))
AllPointingData$Integration_start_landmark <- as.numeric(as.matrix(AllPointingData$Integration_start_landmark))
AllPointingData$Integration_target_landmark <- as.numeric(as.matrix(AllPointingData$Integration_target_landmark))
AllPointingData$Choice_start_landmark <- as.numeric(as.matrix(AllPointingData$Choice_start_landmark))
AllPointingData$Choice_target_landmark <- as.numeric(as.matrix(AllPointingData$Choice_target_landmark))

In [50]:
## VGA Analysis predictors 
AllPointingData$VGA_Connectivity_start_landmark <- as.numeric(as.matrix(AllPointingData$VGA_Connectivity_start_landmark))
AllPointingData$VGA_Connectivity_target_landmark <- as.numeric(as.matrix(AllPointingData$VGA_Connectivity_target_landmark))
AllPointingData$VGA_Integration_start_landmark <- as.numeric(as.matrix(AllPointingData$VGA_Integration_start_landmark))
AllPointingData$VGA_Integration_target_landmark <- as.numeric(as.matrix(AllPointingData$VGA_Integration_target_landmark))
AllPointingData$VGA_ThruVision_start_landmark <- as.numeric(as.matrix(AllPointingData$VGA_ThruVision_start_landmark))
AllPointingData$VGA_ThruVision_target_landmark <- as.numeric(as.matrix(AllPointingData$VGA_ThruVision_target_landmark))

In [51]:
# Scale Through Vision measure, given warning that it's on a very different scale 
AllPointingData$VGA_ThruVision_start_landmark <- scale(AllPointingData$VGA_ThruVision_start_landmark)
AllPointingData$VGA_ThruVision_target_landmark <- scale(AllPointingData$VGA_ThruVision_target_landmark)

In [52]:
# Check out distribution of sex across orders
with(AllPointingData, table(sex, Route_Order)) 

   Route_Order
sex    1    2
  1 1501 1059
  2  385  668

In [53]:
################################
## Set-up contrast structure ##
################################


AllPointingData <- within(AllPointingData, {
    # Factor 1: same_or_different
    Route.diffVsame <- ifelse(same_or_different=="different", 1/2, 
                              ifelse( same_or_different=="same", -1/2, NA ) ) #smaller value because we expect smaller error for same route judgments
    # Factor 2: sex
    sex.fVm <- ifelse(sex=="1", 1/2, #female
                      ifelse( sex=="2", -1/2, NA ) ) #male
    # Factor 3: RouteOrder
    RouteOrder.1V2 <- ifelse(Route_Order=="1", 1/2, 
                             ifelse( Route_Order=="2", -1/2, NA ) ) 
})

In [54]:
# LMEM from previous submission
#AbsError.model = lmer(abs_error ~ Route.diffVsame*sex.fVm*RouteOrder.1V2 + (1 + Route.diffVsame | participant),  
#                      data = AllPointingData)
#coef(AbsError.model)
#summary(AbsError.model)

# Get effect size for model
#r.squaredGLMM(AbsError.model)[1]
#r.squaredGLMM(AbsError.model)[2] #we report this

### NEW: Model with Connectivity [Axial Map] ###

AbsError_w_Connectivity.model = lmer(abs_error ~ Route.diffVsame*sex.fVm*RouteOrder.1V2 
                                        + Connectivity_start_landmark 
                                        + Connectivity_target_landmark 
                                        + (1 + Route.diffVsame | participant),  
                                        data = AllPointingData)
#coef(AbsError_w_Connectivity.model)
summary(AbsError_w_Connectivity.model)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: 
abs_error ~ Route.diffVsame * sex.fVm * RouteOrder.1V2 + Connectivity_start_landmark +  
    Connectivity_target_landmark + (1 + Route.diffVsame | participant)
   Data: AllPointingData

REML criterion at convergence: 36213.9

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.9943 -0.6474 -0.2305  0.3888  4.2095 

Random effects:
 Groups      Name            Variance Std.Dev. Corr
 participant (Intercept)       90.53   9.514       
             Route.diffVsame  126.62  11.252   0.04
 Residual                    1278.09  35.750       
Number of obs: 3613, groups:  participant, 65

Fixed effects:
                                        Estimate Std. Error        df t value
(Intercept)                              52.2175     2.5392  471.5665  20.565
Route.diffVsame                          18.9829     2.0894   61.1258   9.085
sex.fVm                                   5.0660     3.0

In [55]:
# Get effect size for model
r.squaredGLMM(AbsError_w_Connectivity.model)[1]
r.squaredGLMM(AbsError_w_Connectivity.model)[2] #we report this

Warning message:
“'r.squaredGLMM' now calculates a revised statistic. See the help page.”


[1] 0.1053284

[1] 0.1837926

In [56]:
### NEW: Model with Integration [Axial Map] ###

AbsError_w_Integration.model = lmer(abs_error ~ Route.diffVsame*sex.fVm*RouteOrder.1V2 
                                     + Integration_start_landmark 
                                     + Integration_target_landmark 
                                     + (1 + Route.diffVsame | participant),  
                                     data = AllPointingData)
#coef(AbsError_w_Integration.model)
summary(AbsError_w_Integration.model)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: 
abs_error ~ Route.diffVsame * sex.fVm * RouteOrder.1V2 + Integration_start_landmark +  
    Integration_target_landmark + (1 + Route.diffVsame | participant)
   Data: AllPointingData

REML criterion at convergence: 36216.7

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.1036 -0.6422 -0.2363  0.4003  4.1537 

Random effects:
 Groups      Name            Variance Std.Dev. Corr
 participant (Intercept)       90.44   9.51        
             Route.diffVsame  126.43  11.24    0.04
 Residual                    1281.94  35.80        
Number of obs: 3613, groups:  participant, 65

Fixed effects:
                                       Estimate Std. Error       df t value
(Intercept)                              52.450      4.687 2478.143  11.190
Route.diffVsame                          18.992      2.090   61.127   9.088
sex.fVm                                   5.044      3.002   61.

In [57]:
# Get effect size for model
r.squaredGLMM(AbsError_w_Integration.model)[1]
r.squaredGLMM(AbsError_w_Integration.model)[2] #we report this

[1] 0.1029607

[1] 0.1813302

In [58]:
### NEW: Model with Choice [Axial Map] ###

AbsError_w_Choice.model = lmer(abs_error ~ Route.diffVsame*sex.fVm*RouteOrder.1V2 
                                    + Choice_start_landmark 
                                    + Choice_target_landmark 
                                    + (1 + Route.diffVsame | participant),  
                                    data = AllPointingData)
#coef(AbsError_w_Integration.model)
summary(AbsError_w_Choice.model)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: 
abs_error ~ Route.diffVsame * sex.fVm * RouteOrder.1V2 + Choice_start_landmark +  
    Choice_target_landmark + (1 + Route.diffVsame | participant)
   Data: AllPointingData

REML criterion at convergence: 36379.4

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.7883 -0.6495 -0.2812  0.3849  4.1474 

Random effects:
 Groups      Name            Variance Std.Dev. Corr
 participant (Intercept)       89.62   9.467       
             Route.diffVsame  122.19  11.054   0.05
 Residual                    1334.61  36.532       
Number of obs: 3613, groups:  participant, 65

Fixed effects:
                                         Estimate Std. Error         df t value
(Intercept)                             3.789e+01  1.837e+00  1.361e+02  20.628
Route.diffVsame                         1.900e+01  2.088e+00  6.114e+01   9.099
sex.fVm                                 5.050e+00  3.004e+00  

In [59]:
# Get effect size for model
r.squaredGLMM(AbsError_w_Choice.model)[1]
r.squaredGLMM(AbsError_w_Choice.model)[2] #we report this

[1] 0.07055432

[1] 0.1477438

In [60]:
### NEW: Model with Connectivity [VGA] ###

AbsError_w_VGA_Connectivity.model = lmer(abs_error ~ Route.diffVsame*sex.fVm*RouteOrder.1V2 
                                     + VGA_Connectivity_start_landmark 
                                     + VGA_Connectivity_target_landmark 
                                     + (1 + Route.diffVsame | participant),  
                                     data = AllPointingData)
#coef(AbsError_w_VGA_Connectivity.model)
summary(AbsError_w_VGA_Connectivity.model)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: 
abs_error ~ Route.diffVsame * sex.fVm * RouteOrder.1V2 + VGA_Connectivity_start_landmark +  
    VGA_Connectivity_target_landmark + (1 + Route.diffVsame |      participant)
   Data: AllPointingData

REML criterion at convergence: 36378.5

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.7418 -0.6524 -0.2812  0.4019  4.1220 

Random effects:
 Groups      Name            Variance Std.Dev. Corr
 participant (Intercept)       89.53   9.462       
             Route.diffVsame  122.00  11.046   0.05
 Residual                    1336.67  36.560       
Number of obs: 3613, groups:  participant, 65

Fixed effects:
                                         Estimate Std. Error         df t value
(Intercept)                              19.64293    5.46300 2950.18406   3.596
Route.diffVsame                          18.98750    2.08808   61.14130   9.093
sex.fVm                              

In [61]:
# Get effect size for model
r.squaredGLMM(AbsError_w_VGA_Connectivity.model)[1]
r.squaredGLMM(AbsError_w_VGA_Connectivity.model)[2] #we report this

[1] 0.06929575

[1] 0.146392

In [62]:
### NEW: Model with visual Integration [VGA] ###

AbsError_w_VGA_Integration.model = lmer(abs_error ~ Route.diffVsame*sex.fVm*RouteOrder.1V2 
                                         + VGA_Integration_start_landmark 
                                         + VGA_Integration_target_landmark 
                                         + (1 + Route.diffVsame | participant),  
                                         data = AllPointingData)
#coef(AbsError_w_VGA_Integration.model)
summary(AbsError_w_VGA_Integration.model)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: 
abs_error ~ Route.diffVsame * sex.fVm * RouteOrder.1V2 + VGA_Integration_start_landmark +  
    VGA_Integration_target_landmark + (1 + Route.diffVsame |      participant)
   Data: AllPointingData

REML criterion at convergence: 36118

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.3099 -0.6693 -0.2173  0.4219  4.1873 

Random effects:
 Groups      Name            Variance Std.Dev. Corr
 participant (Intercept)       90.94   9.536       
             Route.diffVsame  128.21  11.323   0.04
 Residual                    1246.31  35.303       
Number of obs: 3613, groups:  participant, 65

Fixed effects:
                                       Estimate Std. Error       df t value
(Intercept)                               8.084      9.084 3521.363   0.890
Route.diffVsame                          19.026      2.086   61.130   9.122
sex.fVm                                   5.052      

In [63]:
# Get effect size for model
r.squaredGLMM(AbsError_w_VGA_Integration.model)[1]
r.squaredGLMM(AbsError_w_VGA_Integration.model)[2] #we report this

[1] 0.1249879

[1] 0.2039711

In [64]:
### NEW [Third submission to SCC - August 2019]: Model with visual Integration [VGA] ###

AbsError_w_VGA_ThruVision.model = lmer(abs_error ~ Route.diffVsame*sex.fVm*RouteOrder.1V2 
                                        + VGA_ThruVision_start_landmark 
                                        + VGA_ThruVision_target_landmark 
                                        + (1 + Route.diffVsame | participant),  
                                        data = AllPointingData)
#coef(AbsError_w_VGA_ThruVision.model)
summary(AbsError_w_VGA_ThruVision.model)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: 
abs_error ~ Route.diffVsame * sex.fVm * RouteOrder.1V2 + VGA_ThruVision_start_landmark +  
    VGA_ThruVision_target_landmark + (1 + Route.diffVsame | participant)
   Data: AllPointingData

REML criterion at convergence: 36298.3

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.9980 -0.6604 -0.2448  0.4174  4.1262 

Random effects:
 Groups      Name            Variance Std.Dev. Corr
 participant (Intercept)       90.0    9.487       
             Route.diffVsame  124.1   11.141   0.05
 Residual                    1310.4   36.199       
Number of obs: 3613, groups:  participant, 65

Fixed effects:
                                        Estimate Std. Error        df t value
(Intercept)                              37.2501     1.5017   61.1316  24.806
Route.diffVsame                          18.9905     2.0889   61.1361   9.091
sex.fVm                                   5.0569    

In [65]:
# Get effect size for model
r.squaredGLMM(AbsError_w_VGA_ThruVision.model)[1]
r.squaredGLMM(AbsError_w_VGA_ThruVision.model)[2] #we report this

[1] 0.08548014

[1] 0.163206

In [66]:
###############################################################################
###### Models with interaction term between start and target - Aug 2019 ######


### Connectivity [Axial Map] ###


AbsError_w_Connectivity.model2 = lmer(abs_error ~ Route.diffVsame*sex.fVm*RouteOrder.1V2 
                                     + Connectivity_start_landmark*Connectivity_target_landmark 
                                     + (1 + Route.diffVsame | participant),  
                                     data = AllPointingData)
#coef(AbsError_w_Connectivity.model2)
summary(AbsError_w_Connectivity.model2)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: 
abs_error ~ Route.diffVsame * sex.fVm * RouteOrder.1V2 + Connectivity_start_landmark *  
    Connectivity_target_landmark + (1 + Route.diffVsame | participant)
   Data: AllPointingData

REML criterion at convergence: 36214.5

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.9745 -0.6458 -0.2320  0.3950  4.2158 

Random effects:
 Groups      Name            Variance Std.Dev. Corr
 participant (Intercept)       90.53   9.514       
             Route.diffVsame  126.62  11.253   0.04
 Residual                    1278.24  35.752       
Number of obs: 3613, groups:  participant, 65

Fixed effects:
                                                          Estimate Std. Error
(Intercept)                                                55.0093     4.3887
Route.diffVsame                                            19.5374     2.2071
sex.fVm                                                 

In [67]:
# Get effect size for model
r.squaredGLMM(AbsError_w_Connectivity.model2)[1]
r.squaredGLMM(AbsError_w_Connectivity.model2)[2] #we report this

[1] 0.1054418

[1] 0.1838873

In [68]:
### Integration [Axial Map] ###

AbsError_w_Integration.model2 = lmer(abs_error ~ Route.diffVsame*sex.fVm*RouteOrder.1V2 
                                    + Integration_start_landmark*Integration_target_landmark 
                                    + (1 + Route.diffVsame | participant),  
                                    data = AllPointingData)
#coef(AbsError_w_Integration.model2)
summary(AbsError_w_Integration.model2)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: 
abs_error ~ Route.diffVsame * sex.fVm * RouteOrder.1V2 + Integration_start_landmark *  
    Integration_target_landmark + (1 + Route.diffVsame | participant)
   Data: AllPointingData

REML criterion at convergence: 36209.2

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.0663 -0.6482 -0.2408  0.3992  4.1625 

Random effects:
 Groups      Name            Variance Std.Dev. Corr
 participant (Intercept)       90.43   9.51        
             Route.diffVsame  126.47  11.25    0.04
 Residual                    1281.98  35.80        
Number of obs: 3613, groups:  participant, 65

Fixed effects:
                                                       Estimate Std. Error
(Intercept)                                              69.904     19.016
Route.diffVsame                                          19.721      2.227
sex.fVm                                                   5.042    

In [69]:
# Get effect size for model
r.squaredGLMM(AbsError_w_Integration.model2)[1]
r.squaredGLMM(AbsError_w_Integration.model2)[2] #we report this

[1] 0.1031409

[1] 0.1814936

In [70]:
### Choice [Axial Map] ###

AbsError_w_Choice.model2 = lmer(abs_error ~ Route.diffVsame*sex.fVm*RouteOrder.1V2 
                               + Choice_start_landmark*Choice_target_landmark 
                               + (1 + Route.diffVsame | participant),  
                               data = AllPointingData)
#coef(AbsError_w_Choice.model2)
summary(AbsError_w_Choice.model2)

Warning message:
“Some predictor variables are on very different scales: consider rescaling”
Warning message:
“Some predictor variables are on very different scales: consider rescaling”


Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: 
abs_error ~ Route.diffVsame * sex.fVm * RouteOrder.1V2 + Choice_start_landmark *  
    Choice_target_landmark + (1 + Route.diffVsame | participant)
   Data: AllPointingData

REML criterion at convergence: 36392.1

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.8510 -0.6472 -0.2792  0.3880  4.1632 

Random effects:
 Groups      Name            Variance Std.Dev. Corr
 participant (Intercept)       89.67   9.469       
             Route.diffVsame  122.31  11.059   0.05
 Residual                    1333.64  36.519       
Number of obs: 3613, groups:  participant, 65

Fixed effects:
                                               Estimate Std. Error         df
(Intercept)                                   3.632e+01  2.021e+00  1.980e+02
Route.diffVsame                               1.920e+01  2.091e+00  6.146e+01
sex.fVm                                       5.051e+00  3.005e+00  

In [71]:
# Get effect size for model
r.squaredGLMM(AbsError_w_Choice.model2)[1]
r.squaredGLMM(AbsError_w_Choice.model2)[2] #we report this

[1] 0.07136248

[1] 0.1485856

In [72]:
### Connectivity [VGA] ###

AbsError_w_VGA_Connectivity.model2 = lmer(abs_error ~ Route.diffVsame*sex.fVm*RouteOrder.1V2 
                                         + VGA_Connectivity_start_landmark*VGA_Connectivity_target_landmark 
                                         + (1 + Route.diffVsame | participant),  
                                         data = AllPointingData)
#coef(AbsError_w_VGA_Connectivity.model2)
summary(AbsError_w_VGA_Connectivity.model2)

Warning message:
“Some predictor variables are on very different scales: consider rescaling”
Warning message:
“Some predictor variables are on very different scales: consider rescaling”


Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: 
abs_error ~ Route.diffVsame * sex.fVm * RouteOrder.1V2 + VGA_Connectivity_start_landmark *  
    VGA_Connectivity_target_landmark + (1 + Route.diffVsame |      participant)
   Data: AllPointingData

REML criterion at convergence: 36388.6

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.7406 -0.6519 -0.2815  0.4010  4.1206 

Random effects:
 Groups      Name            Variance Std.Dev. Corr
 participant (Intercept)       89.52   9.462       
             Route.diffVsame  121.96  11.044   0.05
 Residual                    1337.05  36.566       
Number of obs: 3613, groups:  participant, 65

Fixed effects:
                                                                   Estimate
(Intercept)                                                       1.788e+01
Route.diffVsame                                                   1.900e+01
sex.fVm                                          

In [73]:
# Get effect size for model
r.squaredGLMM(AbsError_w_VGA_Connectivity.model2)[1]
r.squaredGLMM(AbsError_w_VGA_Connectivity.model2)[2] #we report this

[1] 0.06928125

[1] 0.1463461

In [74]:
### Integration [VGA] ###

AbsError_w_VGA_Integration.model2 = lmer(abs_error ~ Route.diffVsame*sex.fVm*RouteOrder.1V2 
                                        + VGA_Integration_start_landmark*VGA_Integration_target_landmark 
                                        + (1 + Route.diffVsame | participant),  
                                        data = AllPointingData)
#coef(AbsError_w_VGA_Integration.model2)
summary(AbsError_w_VGA_Integration.model2)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: 
abs_error ~ Route.diffVsame * sex.fVm * RouteOrder.1V2 + VGA_Integration_start_landmark *  
    VGA_Integration_target_landmark + (1 + Route.diffVsame |      participant)
   Data: AllPointingData

REML criterion at convergence: 36100.5

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.4621 -0.6602 -0.2133  0.4251  4.1651 

Random effects:
 Groups      Name            Variance Std.Dev. Corr
 participant (Intercept)       91.05   9.542       
             Route.diffVsame  128.46  11.334   0.04
 Residual                    1242.83  35.254       
Number of obs: 3613, groups:  participant, 65

Fixed effects:
                                                               Estimate
(Intercept)                                                    -234.381
Route.diffVsame                                                  16.888
sex.fVm                                                        

In [75]:
# Get effect size for model
r.squaredGLMM(AbsError_w_VGA_Integration.model2)[1]
r.squaredGLMM(AbsError_w_VGA_Integration.model2)[2] #we report this

[1] 0.1273104

[1] 0.2063923

In [76]:
### Through Vision [VGA] ###

AbsError_w_VGA_ThruVision.model2 = lmer(abs_error ~ Route.diffVsame*sex.fVm*RouteOrder.1V2 
                                       + VGA_ThruVision_start_landmark*VGA_ThruVision_target_landmark 
                                       + (1 + Route.diffVsame | participant),  
                                       data = AllPointingData)
#coef(AbsError_w_VGA_ThruVision.model2)
summary(AbsError_w_VGA_ThruVision.model2)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: 
abs_error ~ Route.diffVsame * sex.fVm * RouteOrder.1V2 + VGA_ThruVision_start_landmark *  
    VGA_ThruVision_target_landmark + (1 + Route.diffVsame | participant)
   Data: AllPointingData

REML criterion at convergence: 36267

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.3003 -0.6573 -0.2456  0.4350  4.1269 

Random effects:
 Groups      Name            Variance Std.Dev. Corr
 participant (Intercept)       90.38   9.507       
             Route.diffVsame  125.22  11.190   0.05
 Residual                    1299.95  36.055       
Number of obs: 3613, groups:  participant, 65

Fixed effects:
                                                              Estimate
(Intercept)                                                    38.5677
Route.diffVsame                                                17.3907
sex.fVm                                                         5.0566
Rout

In [77]:
# Get effect size for model
r.squaredGLMM(AbsError_w_VGA_ThruVision.model2)[1]
r.squaredGLMM(AbsError_w_VGA_ThruVision.model2)[2] #we report this

[1] 0.09202581

[1] 0.1701494

In [78]:
#######################################
#### Exploration with visibility ######
#######################################

# We need to create a factor that distinguishes: same-seen, same-unseen, and different
AllPointingData$visibility = 'different'
AllPointingData$visibility[AllPointingData$same_or_different %in% c('same') & AllPointingData$seenunseen %in% c('0')]='sameseen'
AllPointingData$visibility[AllPointingData$same_or_different %in% c('same') & AllPointingData$seenunseen %in% c('1')]='sameunseen'

pander(aggregate(abs_error~visibility,data=AllPointingData,FUN=mean))



| visibility | abs_error |
|:----------:|:---------:|
| different  |   48.3    |
|  sameseen  |   23.63   |
| sameunseen |   40.31   |



In [79]:
pander(aggregate(abs_error~visibility,data=AllPointingData,FUN=sd))



| visibility | abs_error |
|:----------:|:---------:|
| different  |   41.75   |
|  sameseen  |   31.57   |
| sameunseen |   33.09   |



In [80]:
# Alternatively, with dplyer
summtError = dplyr::summarize(group_by(AllPointingData,visibility),
                              mean=mean(abs_error, na.rm = TRUE), sd= sd(abs_error, na.rm = TRUE), se=sd(abs_error, na.rm = TRUE)/sqrt(length(abs_error)))
print.data.frame(summtError)

  visibility     mean       sd        se
1  different 48.30185 41.74828 0.9182657
2   sameseen 23.63240 31.56828 0.9544261
3 sameunseen 40.30722 33.08799 1.5563280


In [81]:
#Define contrast structure for this factor
AllPointingData2 <- within(AllPointingData, {
  
  # Factor 1: Visibility type
  Visibility.sameseenVsameunseen <- ifelse( visibility=="different", 0,
                        ifelse( visibility=="sameunseen", 1/2,
                                ifelse( visibility=="sameseen", -1/2, NA )))
  
  Visibility.sameseenVdifferent <- ifelse( visibility=="different", 1/2,
                                            ifelse( visibility=="sameunseen", 0,
                                                    ifelse( visibility=="sameseen", -1/2, NA )))
  # Factor 2: sex
  sex.fVm <- ifelse( sex=="1", 1/2, #female
                     ifelse( sex=="2", -1/2, NA ) ) #male
  
  
  # Factor 3: RouteOrder
  RouteOrder.1V2 <- ifelse( Route_Order=="1", 1/2, 
                            ifelse( Route_Order=="2", -1/2, NA ) ) 
  
} )

In [82]:
# LMEMs from previous submission
#AbsError_SeenUnseen.model = lmer(abs_error ~ (Visibility.sameseenVsameunseen+Visibility.sameseenVdifferent)*sex.fVm*RouteOrder.1V2 
#                                 + (1 + (Visibility.sameseenVsameunseen+Visibility.sameseenVdifferent) | participant),  
#                       data = AllPointingData2)
#coef(AbsError_SeenUnseen.model)
#summary(AbsError_SeenUnseen.model)

# Get effect size for model
#r.squaredGLMM(AbsError_SeenUnseen.model)[1]
#r.squaredGLMM(AbsError_SeenUnseen.model)[2] #we report this


### NEW: Model with Connectivity [Axial Map] ###
AbsError_SeenUnseen_Connectivity.model = lmer(abs_error ~ (Visibility.sameseenVsameunseen+Visibility.sameseenVdifferent)*sex.fVm*RouteOrder.1V2  
                                   + Connectivity_start_landmark
                                  + Connectivity_target_landmark
                                  + (1 + (Visibility.sameseenVsameunseen+Visibility.sameseenVdifferent) | participant),  
                                 data = AllPointingData2)
#coef(AbsError_SeenUnseen_Connectivity.model)
summary(AbsError_SeenUnseen_Connectivity.model)

boundary (singular) fit: see ?isSingular


Correlation matrix not shown by default, as p = 14 > 12.
Use print(obj, correlation=TRUE)  or
    vcov(obj)        if you need it




Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: 
abs_error ~ (Visibility.sameseenVsameunseen + Visibility.sameseenVdifferent) *  
    sex.fVm * RouteOrder.1V2 + Connectivity_start_landmark +  
    Connectivity_target_landmark + (1 + (Visibility.sameseenVsameunseen +  
    Visibility.sameseenVdifferent) | participant)
   Data: AllPointingData2

REML criterion at convergence: 36157.4

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.0090 -0.6473 -0.2216  0.3685  4.1902 

Random effects:
 Groups      Name                           Variance Std.Dev. Corr       
 participant (Intercept)                      89.72   9.472              
             Visibility.sameseenVsameunseen   67.36   8.208    0.00      
             Visibility.sameseenVdifferent   230.10  15.169   -0.14 -0.99
 Residual                                   1266.74  35.591              
Number of obs: 3613, groups:  participant, 65

Fixed effects:
                 

In [83]:
# Get effect size for model
r.squaredGLMM(AbsError_SeenUnseen_Connectivity.model)[1]
r.squaredGLMM(AbsError_SeenUnseen_Connectivity.model)[2] #we report this

[1] 0.1130691

[1] 0.1917092

In [84]:
### NEW: Model with Integration [Axial Map] ###
AbsError_SeenUnseen_Integration.model = lmer(abs_error ~ (Visibility.sameseenVsameunseen+Visibility.sameseenVdifferent)*sex.fVm*RouteOrder.1V2  
                                              + Integration_start_landmark
                                              + Integration_target_landmark
                                              + (1 + (Visibility.sameseenVsameunseen+Visibility.sameseenVdifferent) | participant),  
                                              data = AllPointingData2)
#coef(AbsError_SeenUnseen_Integration.model)
summary(AbsError_SeenUnseen_Integration.model)

boundary (singular) fit: see ?isSingular


Correlation matrix not shown by default, as p = 14 > 12.
Use print(obj, correlation=TRUE)  or
    vcov(obj)        if you need it




Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: 
abs_error ~ (Visibility.sameseenVsameunseen + Visibility.sameseenVdifferent) *  
    sex.fVm * RouteOrder.1V2 + Integration_start_landmark + Integration_target_landmark +  
    (1 + (Visibility.sameseenVsameunseen + Visibility.sameseenVdifferent) |  
        participant)
   Data: AllPointingData2

REML criterion at convergence: 36146.1

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.1170 -0.6402 -0.2243  0.3775  3.9827 

Random effects:
 Groups      Name                           Variance Std.Dev. Corr       
 participant (Intercept)                      89.72   9.472              
             Visibility.sameseenVsameunseen   67.36   8.207    0.00      
             Visibility.sameseenVdifferent   230.18  15.172   -0.14 -0.99
 Residual                                   1265.44  35.573              
Number of obs: 3613, groups:  participant, 65

Fixed effects:
               

In [85]:
# Get effect size for model
r.squaredGLMM(AbsError_SeenUnseen_Integration.model)[1]
r.squaredGLMM(AbsError_SeenUnseen_Integration.model)[2] #we report this

[1] 0.113886

[1] 0.1925233

In [86]:
### NEW: Model with Choice [Axial Map] ###
AbsError_SeenUnseen_Choice.model = lmer(abs_error ~ (Visibility.sameseenVsameunseen+Visibility.sameseenVdifferent)*sex.fVm*RouteOrder.1V2  
                                             + Choice_start_landmark
                                             + Choice_target_landmark
                                             + (1 + (Visibility.sameseenVsameunseen+Visibility.sameseenVdifferent) | participant),  
                                             data = AllPointingData2)
#coef(AbsError_SeenUnseen_Choice.model)
summary(AbsError_SeenUnseen_Choice.model)


Correlation matrix not shown by default, as p = 14 > 12.
Use print(obj, correlation=TRUE)  or
    vcov(obj)        if you need it




Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: 
abs_error ~ (Visibility.sameseenVsameunseen + Visibility.sameseenVdifferent) *  
    sex.fVm * RouteOrder.1V2 + Choice_start_landmark + Choice_target_landmark +  
    (1 + (Visibility.sameseenVsameunseen + Visibility.sameseenVdifferent) |  
        participant)
   Data: AllPointingData2

REML criterion at convergence: 36292.4

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.7928 -0.6438 -0.2561  0.3567  4.0192 

Random effects:
 Groups      Name                           Variance Std.Dev. Corr       
 participant (Intercept)                      88.86   9.427              
             Visibility.sameseenVsameunseen   65.44   8.090   -0.01      
             Visibility.sameseenVdifferent   223.60  14.953   -0.13 -0.99
 Residual                                   1311.21  36.211              
Number of obs: 3613, groups:  participant, 65

Fixed effects:
                         

In [87]:
# Get effect size for model
r.squaredGLMM(AbsError_SeenUnseen_Choice.model)[1]
r.squaredGLMM(AbsError_SeenUnseen_Choice.model)[2] #we report this

[1] 0.08576846

[1] 0.1633826

In [88]:
### NEW: Model with Connectivity [VGA] ###
AbsError_SeenUnseen_VGA_Connectivity.model = lmer(abs_error ~ (Visibility.sameseenVsameunseen+Visibility.sameseenVdifferent)*sex.fVm*RouteOrder.1V2  
                                              + VGA_Connectivity_start_landmark
                                              + VGA_Connectivity_target_landmark
                                              + (1 + (Visibility.sameseenVsameunseen+Visibility.sameseenVdifferent) | participant),  
                                              data = AllPointingData2)
#coef(AbsError_SeenUnseen_VGA_Connectivity.model)
summary(AbsError_SeenUnseen_VGA_Connectivity.model)

boundary (singular) fit: see ?isSingular


Correlation matrix not shown by default, as p = 14 > 12.
Use print(obj, correlation=TRUE)  or
    vcov(obj)        if you need it




Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: 
abs_error ~ (Visibility.sameseenVsameunseen + Visibility.sameseenVdifferent) *  
    sex.fVm * RouteOrder.1V2 + VGA_Connectivity_start_landmark +  
    VGA_Connectivity_target_landmark + (1 + (Visibility.sameseenVsameunseen +  
    Visibility.sameseenVdifferent) | participant)
   Data: AllPointingData2

REML criterion at convergence: 36281.9

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.7799 -0.6484 -0.2546  0.3586  3.9841 

Random effects:
 Groups      Name                           Variance Std.Dev. Corr       
 participant (Intercept)                      88.93   9.430              
             Visibility.sameseenVsameunseen   65.33   8.083    0.00      
             Visibility.sameseenVdifferent   223.60  14.953   -0.14 -0.99
 Residual                                   1309.63  36.189              
Number of obs: 3613, groups:  participant, 65

Fixed effects:
         

In [89]:
# Get effect size for model
r.squaredGLMM(AbsError_SeenUnseen_VGA_Connectivity.model)[1]
r.squaredGLMM(AbsError_SeenUnseen_VGA_Connectivity.model)[2] #we report this

[1] 0.08675291

[1] 0.1643507

In [90]:
### NEW: Model with Integration [VGA] ###
AbsError_SeenUnseen_VGA_Integration.model = lmer(abs_error ~ (Visibility.sameseenVsameunseen+Visibility.sameseenVdifferent)*sex.fVm*RouteOrder.1V2  
                                                  + VGA_Integration_start_landmark
                                                  + VGA_Integration_target_landmark
                                                  + (1 + (Visibility.sameseenVsameunseen+Visibility.sameseenVdifferent) | participant),  
                                                  data = AllPointingData2)
#coef(AbsError_SeenUnseen_VGA_Integration.model)
summary(AbsError_SeenUnseen_VGA_Integration.model)

boundary (singular) fit: see ?isSingular


Correlation matrix not shown by default, as p = 14 > 12.
Use print(obj, correlation=TRUE)  or
    vcov(obj)        if you need it




Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: 
abs_error ~ (Visibility.sameseenVsameunseen + Visibility.sameseenVdifferent) *  
    sex.fVm * RouteOrder.1V2 + VGA_Integration_start_landmark +  
    VGA_Integration_target_landmark + (1 + (Visibility.sameseenVsameunseen +  
    Visibility.sameseenVdifferent) | participant)
   Data: AllPointingData2

REML criterion at convergence: 36008.2

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.3155 -0.6647 -0.2012  0.4220  3.9141 

Random effects:
 Groups      Name                           Variance Std.Dev. Corr       
 participant (Intercept)                      90.35   9.505              
             Visibility.sameseenVsameunseen   69.27   8.323   -0.01      
             Visibility.sameseenVdifferent   234.89  15.326   -0.14 -0.99
 Residual                                   1216.46  34.878              
Number of obs: 3613, groups:  participant, 65

Fixed effects:
           

In [91]:
# Get effect size for model
r.squaredGLMM(AbsError_SeenUnseen_VGA_Integration.model)[1]
r.squaredGLMM(AbsError_SeenUnseen_VGA_Integration.model)[2] #we report this

[1] 0.144098

[1] 0.2236164

In [92]:
### NEW: Model with Through Vision [VGA] - Aug 2019 ###
AbsError_SeenUnseen_VGA_ThruVision.model <- lmer(abs_error ~ (Visibility.sameseenVsameunseen+Visibility.sameseenVdifferent)*sex.fVm*RouteOrder.1V2  
                                                 + VGA_ThruVision_start_landmark 
                                                 + VGA_ThruVision_target_landmark  
                                                 + (1 + (Visibility.sameseenVsameunseen+Visibility.sameseenVdifferent) | participant),  
                                                 data = AllPointingData2)
#coef(AbsError_SeenUnseen_VGA_ThruVision.model)
summary(AbsError_SeenUnseen_VGA_ThruVision.model)

boundary (singular) fit: see ?isSingular


Correlation matrix not shown by default, as p = 14 > 12.
Use print(obj, correlation=TRUE)  or
    vcov(obj)        if you need it




Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: 
abs_error ~ (Visibility.sameseenVsameunseen + Visibility.sameseenVdifferent) *  
    sex.fVm * RouteOrder.1V2 + VGA_ThruVision_start_landmark +  
    VGA_ThruVision_target_landmark + (1 + (Visibility.sameseenVsameunseen +  
    Visibility.sameseenVdifferent) | participant)
   Data: AllPointingData2

REML criterion at convergence: 36192

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.0936 -0.6525 -0.2189  0.3956  3.8151 

Random effects:
 Groups      Name                           Variance Std.Dev. Corr       
 participant (Intercept)                      89.58   9.465              
             Visibility.sameseenVsameunseen   66.28   8.141    0.00      
             Visibility.sameseenVdifferent   227.54  15.084   -0.14 -0.99
 Residual                                   1280.31  35.781              
Number of obs: 3613, groups:  participant, 65

Fixed effects:
               

In [93]:
# Get effect size for model
r.squaredGLMM(AbsError_SeenUnseen_VGA_ThruVision.model)[1]
r.squaredGLMM(AbsError_SeenUnseen_VGA_ThruVision.model)[2] #we report this

[1] 0.1047757

[1] 0.1830351

In [94]:
########################################
##### Model Building LMEMs #############
########################################

BySubsData <- within(BySubsData, {
  
  # Factor 1: sex
  sex.fVm <- ifelse( sex=="1", 1/2, #female
                     ifelse( sex=="2", -1/2, NA ) ) #male
  
  # Factor 2: RouteOrder
  RouteOrder.1V2 <- ifelse( Route_Order=="1", 1/2, 
                            ifelse( Route_Order=="2", -1/2, NA ) ) 
  
} )

Rsquared_Overall.model = lm(r2_Overall ~ sex.fVm*RouteOrder.1V2, data = BySubsData)
summary(Rsquared_Overall.model) # this is reported in the manuscript


Call:
lm(formula = r2_Overall ~ sex.fVm * RouteOrder.1V2, data = BySubsData)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.35860 -0.17537 -0.04322  0.12993  0.55376 

Coefficients:
                        Estimate Std. Error t value Pr(>|t|)    
(Intercept)             0.430283   0.033830  12.719   <2e-16 ***
sex.fVm                -0.005462   0.067660  -0.081    0.936    
RouteOrder.1V2         -0.030255   0.067660  -0.447    0.656    
sex.fVm:RouteOrder.1V2  0.007726   0.135319   0.057    0.955    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2408 on 61 degrees of freedom
Multiple R-squared:  0.004204,	Adjusted R-squared:  -0.04477 
F-statistic: 0.08585 on 3 and 61 DF,  p-value: 0.9675


In [95]:
# For a system with landmarks from Route A only
Rsquared_RouteA.model1 = lm(r2_RouteA ~ sex.fVm*RouteOrder.1V2, data = BySubsData)
summary(Rsquared_RouteA.model1)


Call:
lm(formula = r2_RouteA ~ sex.fVm * RouteOrder.1V2, data = BySubsData)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.51815 -0.18714  0.03283  0.22531  0.47706 

Coefficients:
                       Estimate Std. Error t value Pr(>|t|)    
(Intercept)             0.53731    0.03887  13.823   <2e-16 ***
sex.fVm                 0.02827    0.07774   0.364    0.717    
RouteOrder.1V2          0.07955    0.07774   1.023    0.310    
sex.fVm:RouteOrder.1V2 -0.03914    0.15548  -0.252    0.802    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2767 on 61 degrees of freedom
Multiple R-squared:  0.02356,	Adjusted R-squared:  -0.02446 
F-statistic: 0.4906 on 3 and 61 DF,  p-value: 0.6901


In [96]:
# For a system with landmarks from Route B only
Rsquared_RouteB.model1 = lm(r2_RouteB ~ sex.fVm*RouteOrder.1V2, data = BySubsData)
summary(Rsquared_RouteB.model1)


Call:
lm(formula = r2_RouteB ~ sex.fVm * RouteOrder.1V2, data = BySubsData)

Residuals:
    Min      1Q  Median      3Q     Max 
-0.6454 -0.2537  0.0677  0.2713  0.4562 

Coefficients:
                       Estimate Std. Error t value Pr(>|t|)    
(Intercept)             0.60537    0.04420  13.697   <2e-16 ***
sex.fVm                -0.13147    0.08840  -1.487    0.142    
RouteOrder.1V2         -0.02501    0.08840  -0.283    0.778    
sex.fVm:RouteOrder.1V2 -0.10628    0.17679  -0.601    0.550    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.3146 on 61 degrees of freedom
Multiple R-squared:  0.04961,	Adjusted R-squared:  0.00287 
F-statistic: 1.061 on 3 and 61 DF,  p-value: 0.3722


In [97]:
#######################################################
#### Correlations with Individual ability #############
#######################################################

#This grabs SBSOD. I think this is what we should use.
#BySubsData_DVs <- BySubsData[, c(4, 8:13, 39:40, 43:46)]

#This grabs SBSOD-CY
BySubsData_DVs <- BySubsData[, c(5, 8:13, 39:40, 43:46)]

bivariateCors <- cor(BySubsData_DVs, use = "all.obs", method = "pearson" )
round(bivariateCors, 3)

,SBSOD_CY_average,r2_Overall,r2_RouteA,r2_RouteB,abs_error,Abserror_Same,Abserror_Diff,error_start_routeA_SAME,error_start_routeB_SAME,error_start_RouteA_Different,error_target_RouteA_Different,error_start_RouteB_Different,error_target_RouteB_Different
SBSOD_CY_average,1.000,-0.027,0.071,0.163,-0.054,-0.110,0.004,-0.081,-0.109,-0.101,0.011,-0.014,-0.124
r2_Overall,-0.027,1.000,-0.051,0.085,-0.125,-0.085,-0.116,-0.006,-0.120,0.011,-0.135,-0.177,-0.074
r2_RouteA,0.071,-0.051,1.000,0.183,-0.389,-0.388,-0.272,-0.320,-0.362,-0.378,-0.298,-0.340,-0.419
r2_RouteB,0.163,0.085,0.183,1.000,-0.404,-0.465,-0.237,-0.333,-0.466,-0.301,-0.313,-0.405,-0.422
abs_error,-0.054,-0.125,-0.389,-0.404,1.000,0.758,0.876,0.674,0.665,0.821,0.916,0.944,0.835
Abserror_Same,-0.110,-0.085,-0.388,-0.465,0.758,1.000,0.349,0.835,0.919,0.699,0.548,0.671,0.834
Abserror_Diff,0.004,-0.116,-0.272,-0.237,0.876,0.349,1.000,0.350,0.276,0.663,0.911,0.859,0.583
error_start_routeA_SAME,-0.081,-0.006,-0.320,-0.333,0.674,0.835,0.350,1.000,0.555,0.797,0.606,0.494,0.569
error_start_routeB_SAME,-0.109,-0.120,-0.362,-0.466,0.665,0.919,0.276,0.555,1.000,0.485,0.399,0.666,0.853
error_start_RouteA_Different,-0.101,0.011,-0.378,-0.301,0.821,0.699,0.663,0.797,0.485,1.000,0.667,0.588,0.793


In [98]:
# Using rcorr function from package Hmisc for p-values
bivariateCors <- rcorr(as.matrix(BySubsData_DVs))
bivariateCors

                              SBSOD_CY_average r2_Overall r2_RouteA r2_RouteB
SBSOD_CY_average                          1.00      -0.03      0.07      0.16
r2_Overall                               -0.03       1.00     -0.05      0.08
r2_RouteA                                 0.07      -0.05      1.00      0.18
r2_RouteB                                 0.16       0.08      0.18      1.00
abs_error                                -0.05      -0.12     -0.39     -0.40
Abserror_Same                            -0.11      -0.09     -0.39     -0.46
Abserror_Diff                             0.00      -0.12     -0.27     -0.24
error_start_routeA_SAME                  -0.08      -0.01     -0.32     -0.33
error_start_routeB_SAME                  -0.11      -0.12     -0.36     -0.47
error_start_RouteA_Different             -0.10       0.01     -0.38     -0.30
error_target_RouteA_Different             0.01      -0.14     -0.30     -0.31
error_start_RouteB_Different             -0.01      -0.18     -0

In [99]:
###################################################
#### Comparison of SBSOD-CY by Gender #############
###################################################
summtSBSODCY = dplyr::summarize(group_by(BySubsData,sex),
                              mean=mean(SBSOD_CY_average, na.rm = TRUE), sd= sd(SBSOD_CY_average, na.rm = TRUE), 
                              se=sd(SBSOD_CY_average, na.rm = TRUE)/sqrt(length(SBSOD_CY_average)))

print.data.frame(summtSBSODCY)

  sex     mean        sd        se
1   1 4.300000 0.9521905 0.1403928
2   2 4.878947 0.8462853 0.1941512


In [100]:
# Check equality of variance
var.test(SBSOD_CY_average ~ sex, data = BySubsData)

t.test(SBSOD_CY_average ~ sex, data = BySubsData, var.equal = TRUE)


	F test to compare two variances

data:  SBSOD_CY_average by sex
F = 1.2659, num df = 45, denom df = 18, p-value = 0.599
alternative hypothesis: true ratio of variances is not equal to 1
95 percent confidence interval:
 0.5356109 2.6149226
sample estimates:
ratio of variances 
          1.265943 



	Two Sample t-test

data:  SBSOD_CY_average by sex
t = -2.2996, df = 63, p-value = 0.0248
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -1.08204616 -0.07584858
sample estimates:
mean in group 1 mean in group 2 
       4.300000        4.878947 
